In [3]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
%matplotlib inline

import os
import sys
from pathlib import Path
from pprint import pprint
import pickle

from utils import utils, files # sibling package...

In [5]:
experiment = "emg_olympics"
subject = "peter"
path_dictionary = files.build_experiment_path_dict(experiment)
subject_metadata_folder = utils.get_subject_folder(experiment,subject)

data folder:  /mnt/c/Users/spencer/data/emg_olympics


- height of target bar vs height of other bars? (some type of score)
- "shape" of bar data overall? 
  - dimensionality
  - ...?
- search strategy (dynamical mode?)
- 

In [7]:
session_dict = path_dictionary["subjects"][subject]["tasks"]["center_hold"]["sessions"]
session_path_list = []

for session in sorted(list(session_dict.keys()),key=lambda x: int(x.split("_")[-1])):
    session_path_list.append(session_dict[session])


In [9]:
hits = []
misses = []
noholds = []
reach_times = []
for i, behavior_path in enumerate(session_path_list):
    outcome_path = [x for x in behavior_path.iterdir() if "result" in x.name][0]
    behavior = np.genfromtxt(str(outcome_path), delimiter=',', skip_header=1, dtype=None, encoding=None)
    print(behavior)
    outcomes = [x[0] for x in behavior]
    for x in behavior:
        if x[0] == "Hit":
            reach_times.append(x[2])
    uniques, counts = np.unique(outcomes, return_counts=True)
    for outcome, count in zip(uniques, counts):
        if outcome == "Hit":
            hits.append(count)
        elif outcome == "Miss":
            misses.append(count)
        elif outcome == "No Hold":
            noholds.append(count)
        else:
            raise ValueError("Unforeseen outcome in behavior: ", outcome)
        if "Hit" not in outcomes:
            hits.append(0)
        if "Miss" not in outcomes:
            misses.append(0)
        if "No Hold" not in outcomes:
            noholds.append(0)
        
fractions = [h*(100/12) for h in hits]
sessions_idxs = range(len(fractions))
z = np.polyfit(sessions_idxs, fractions, 1)
p = np.poly1d(z)

total_fraction = sum(hits)/(len(session_path_list)*12)
print("hit score: ", total_fraction)

fig, ax = plt.subplots(1,1,figsize=(10,8))
ax.plot(fractions , "r-o", label="hit")
ax.plot(sessions_idxs,p(sessions_idxs) , "k--", label="trend")
# ax.plot([m*(100/12) for m in misses] , "g-o", label="miss")
# ax.plot([n*(100/12) for n in noholds] , "b-o", label="no hold")
# ax.set_xticks([0,1,2])
# ax.set_xticklabels([1,2,3])
ax.set_xlabel("session")
ax.set_ylabel("hit fraction")
ax.set_title("hit fractions over sessions")
ax.set_ylim([0,100])
ax.legend()

fig, ax = plt.subplots(1,1,figsize=(10,8))
ax.plot(reach_times , "b-o", label="reach time")
# ax.plot([m*(100/12) for m in misses] , "g-o", label="miss")
# ax.plot([n*(100/12) for n in noholds] , "b-o", label="no hold")
# ax.set_xticks([0,1,2])
# ax.set_xticklabels([1,2,3])
ax.set_xlabel("session")
ax.set_ylabel("time [s]")
ax.set_title("reach times over sessions")
ax.set_ylim([0,5])
ax.legend()

[('No Hold', 3.516667 ,      nan, -0.87, -0.5 )
 ('Hit', 0.7666667, 1.866667, -0.87, -0.5 )
 ('Miss', 0.7666667, 5.016667, -0.87,  0.5 )
 ('Miss', 0.7666667, 5.016667,  0.  , -1.  )
 ('No Hold', 3.516667 ,      nan, -1.  ,  0.  )
 ('Miss', 0.7666667, 5.016667, -1.  ,  0.  )
 ('Miss', 2.85     , 5.016667, -0.5 , -0.87)
 ('Hit', 1.366667 , 2.566667,  0.87, -0.5 )
 ('Hit', 1.283333 , 0.7     ,  1.  ,  0.  )
 ('Miss', 1.516667 , 5.016667,  0.5 ,  0.87)
 ('Hit', 0.7666667, 3.966667,  0.  ,  1.  )
 ('Miss', 0.7666667, 5.016667, -0.5 ,  0.87)
 ('Miss', 1.1      , 5.016667,  0.5 , -0.87)
 ('Miss', 1.433333 , 5.016667,  0.87,  0.5 )]
[('Hit', 0.7666667, 1.333333,  1.  ,  0.  )
 ('Miss', 0.7833334, 5.016667, -0.87, -0.5 )
 ('Miss', 0.7666667, 5.016667,  0.  , -1.  )
 ('No Hold', 3.516667 ,      nan,  0.87,  0.5 )
 ('No Hold', 3.516667 ,      nan,  0.87,  0.5 )
 ('No Hold', 3.516667 ,      nan,  0.87,  0.5 )
 ('No Hold', 3.516667 ,      nan,  0.87,  0.5 )
 ('Miss', 1.816667 , 5.016667, -1.  ,  0.

IndexError: index 1 is out of bounds for axis 0 with size 1

In [36]:
12*15

180

# PCA

In [ ]:
# get components -- channel-wise covariance
num_components = 3
pca_model = PCA(n_components=num_components)
pca_model.fit(filtered_calibration_data)
pca_components = pca_model.components_
print(pca_components.shape)

In [ ]:
# screeplots
fig, (ax1, ax2) = plt.subplots(1,2)
ax1.plot(range(1,num_components+1), pca_model.explained_variance_ratio_)
ax1.set_ylabel("explained_variance")
ax1.set_xlabel("component")

ax2.plot(range(1,num_components+1), pca_model.singular_values_)
ax2.set_ylabel("singular value")
ax2.set_xlabel("component")

# choose components
fig, axes = plt.subplots(1,num_components,figsize=(18,6))
for i, ax in enumerate(axes):
    ax.imshow(pca_components[i].reshape(8,8),vmin=np.min(pca_components),vmax=np.max(pca_components))
    ax.set_title(str(i))

plt.figure()
plt.plot(pca_components[0]*np.sqrt(pca_model.singular_values_[0]))
plt.plot(movement_vars)

In [ ]:
print(np.dot(components[0],components[1]))
print(np.dot(components[1],components[1]))
print(np.linalg.norm(components[0]))

# factor analysis

https://stackoverflow.com/questions/37221635/rotation-argument-for-scikit-learns-factor-analysis
https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.FastICA.html#sklearn.decomposition.FastICA
https://scikit-learn.org/stable/auto_examples/decomposition/plot_varimax_fa.html#sphx-glr-auto-examples-decomposition-plot-varimax-fa-py
https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.FactorAnalysis.html
https://scikit-learn.org/stable/modules/decomposition.html#fa

In [ ]:
model = FactorAnalysis(n_components=num_components, random_state=0)
model.fit(standardized_data)
factors = model.components_
print(factors.shape)

In [ ]:
print(factors[0].shape)
print(np.dot(factors[0]/np.linalg.norm(factors[0]),
             factors[1]/np.linalg.norm(factors[1])))

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2)
ax1.plot(range(1,65), model.mean_)
ax1.set_ylabel("feature mean")
ax1.set_xlabel("feature")

ax2.plot(range(1,65), model.noise_variance_)
ax2.set_ylabel("feature noise variance")
ax2.set_xlabel("feature")

# choose components
fig, axes = plt.subplots(1,num_components,figsize=(18,6))
for i, ax in enumerate(axes):
    ax.imshow(factors[i].reshape(8,8))
    ax.set_title(str(i))